## Import packages

In [1]:
import selenium
from selenium import webdriver

import shutil, time, os
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

from importlib import reload

import src
reload(src)

<module 'src' from 'C:\\Users\\Matth\\git\\DataAnalysisWorkbooks\\warcraftLogs\\src.py'>

## Configurations

In [2]:
# Set path variables
path_to_adblocker, path_to_download_dir = src.get_path_settings()

In [3]:
# Number of parses to scrape
N_parses = 501
retry_attempts = 3

# Enable/disable SMS
twilio = True

# Enable/disable verbose printouts (debugger)
verbose = True
verbose_rotation = False

## Navigate to WCL

In [4]:
boss = "High Warlord Naj'entus"

In [5]:
# Dictionary for browser to navigate to boss page URLs
boss_link_dict = {"High Warlord Naj'entus" : "#boss=601", "Supremus" : "#boss=602", "Shade of Akama" : "#boss=603", 
                  "Teron Gorefiend" : "#boss=604", "Gurtogg Bloodboil" : "#boss=605", "Reliquary of Souls" : "#boss=606", 
                  "Mother Shahraz" : "#boss=607", "The Illidari Council" : "#boss=608", "Illidan Stormrage" : "#boss=609", 
                  "Rage Winterchill" : "#boss=618", "Anetheron" : "#boss=619", "Kaz'rogal" : "#boss=620", 
                  "Azgalor" : "#boss=621", "Archimonde" : "#boss=622"}

In [6]:
boss_page_url, browser = src.load_top_N_scraper(path_to_adblocker, boss, boss_link_dict)

## Main code loop

In [ ]:
def main():
    
    retry_attempt = 0
    
    # The browser can crash at random due to the page loading too slowly. Rather than restarting the loop manually,
    # use 'retry_attempts' (default = 3) and the following while loop to restart automatically.
    while True: 

        # Twilio is an SMS service that will send a text to your phone when the code finishes/crashes.
        # If you haven't setup twilio then set the flag to False in the Configuration cell.
        if twilio: 
            from twilio.rest import Client
            accountSID, authToken, myTwilioNumber, myCellPhone = src.get_twilio_info()
            twilioCli = Client(accountSID, authToken)

        boss_page_url = f'https://classic.warcraftlogs.com/zone/rankings/1011{boss_link_dict[boss]}&class=Druid&spec=Restoration&metric=hps'
        browser.get(boss_page_url)
        time.sleep(2)

        # First check to see if any of the rankings have changed since last scraping and update the excel file
        print("Checking for rank changes since last scrape...")
        already_recorded_indices = src.get_latest_ranks(browser, boss, boss_link_dict, N_parses)

        print("Rank updates complete.")
        print('-----------')
        time.sleep(2)

        page = 1
        boss_page_url = f'https://classic.warcraftlogs.com/zone/rankings/1011{boss_link_dict[boss]}&class=Druid&spec=Restoration&metric=hps'
        browser.get(boss_page_url)
        time.sleep(2)

        try:
            print("Beginning data scrape...")

            for i in range(1, N_parses):

                # If the player's parse is already in the spreadsheet, skip to the next parse.
                if i in already_recorded_indices: 
                    if i % 100 == 0: 
                        page += 1
                        boss_page_url = f'https://classic.warcraftlogs.com/zone/rankings/1011{boss_link_dict[boss]}&class=Druid&spec=Restoration&metric=hps&page={page}'
                        browser.get(boss_page_url)
                        time.sleep(2) 
                    continue

                rank, name, server, region, date, HPS, duration = src.get_boss_data_top_N_scraper(browser, boss, boss_link_dict, i)
                if name in ['抄能力']: continue  # These players have broken or private reports, skip 

                print(rank, name, server, region)

                link = browser.find_element_by_link_text(name)
                link.click()
                time.sleep(1)

                player_df = pd.DataFrame(pd.np.empty((0, 23)))
                player_df.columns = ["Rank", "Name", "Server", "Date", "Duration", "nHealers", "Spriest?", "Innervate?", "Bloodlust?", "Nature's Grace?", "Power Infusion?", "LB_uptime", "HPS", "% LB (tick) HPS", "% LB (bloom) HPS", "% Rejuv HPS", "% Regrowth HPS", "% Swiftmend HPS", "Rotating on tank?", "Rotation 1", "% Rotation 1", "Rotation 2", "% Rotation 2"]

                temp_url = browser.current_url
                time.sleep(1)

                boss_tanks = src.get_tanks(browser)
                nHealers = src.get_nHealers(browser)

                browser.get(temp_url)
                time.sleep(0.5)

                player_link = browser.find_element_by_link_text(name)
                player_link.click()
                time.sleep(0.5)

                # Scrape HPS data
                LBtick_HPS, LBbloom_HPS, rejuv_HPS, regrowth_HPS, swiftmend_HPS, LB_uptime = src.get_spell_info(browser, HPS)
                time.sleep(2)

                # Check for buffs
                spriest = src.check_spriest(browser)
                innervate, bloodlust, powerInfusion, naturesGrace = src.check_buffs(browser)
                time.sleep(0.5)    

                # Download the cast-sequence CSV.
                src.download_csv(browser, temp_url, "filter-casts-tab", path_to_download_dir, "data/cast_sequence.csv")
                time.sleep(1)

                # Clean the csv
                df = src.clean_cast_sequence_csv()
                df = src.fix_cast_time(df)
                time.sleep(0.5)

                # Get the rotations
                rotation1, rotation1_percent, rotation2, rotation2_percent, rotating_on_tank = src.calculate_rotations(df, boss, boss_tanks, LB_uptime, verbose, verbose_rotation)
                print(f'Rotations: {rotation1} ({rotation1_percent}), {rotation2} ({rotation2_percent})')  

                # Export data and cleanup
                to_append = [rank, name, server + " " + region, date, duration, str(nHealers), spriest, innervate, bloodlust, powerInfusion, naturesGrace, LB_uptime, HPS, LBtick_HPS, LBbloom_HPS, rejuv_HPS, regrowth_HPS, swiftmend_HPS, rotating_on_tank, rotation1, rotation1_percent, rotation2, rotation2_percent]
                src.export_to_excel(boss, to_append, player_df, name, 'top_N_druids', True)

                os.remove(f"data/{boss.replace(' ', '')}_{name}.csv")
                os.remove(f"data/cast_sequence.csv")

                print('-------')
                browser.get(boss_page_url)
                time.sleep(1)

                if i % 100 == 0: 
                    page += 1
                    boss_page_url = f'https://classic.warcraftlogs.com/zone/rankings/1011{boss_link_dict[boss]}&class=Druid&spec=Restoration&metric=hps&page={page}'
                    browser.get(boss_page_url)
                    time.sleep(2) 


            print(f"{boss} scraping complete.")
            print("---------------")

            if twilio: message = twilioCli.messages.create(body = 'Data scraping complete!', from_ = myTwilioNumber, to = myCellPhone)
            break

        except:
            retry_attempt += 1
            time.sleep(10)
            if retry_attempt == retry_attempts: 
                if twilio: message = twilioCli.messages.create(body='Data scraping crashed', from_=myTwilioNumber, to=myCellPhone)
                break
                
                
if __name__ == "__main__":
    main()

Checking for rank changes since last scrape...
50
100
150
200
250
300
350
400
450
500
550
Rank updates complete.
-----------
Beginning data scrape...
457 荷西  寒脊山小径
Lifebloom uptime %: 29.12%
0LB 1I 4RG 0.375
0LB 0I 5RG 0.5
0LB 2I 3RG 0.062
0LB 3I 2RG 0.062
Non-tank rotations %: 1.0
Rotating on tank?: No
Rotations: 0LB 0I 5RG (0.5), 0LB 1I 4RG (0.375)
Removing spreadsheet duplicates..
Duplicate removal complete!
-------
458 香蕉马库斯  法尔班克斯
Lifebloom uptime %: 100.00%
1LB 1I 1RG 0.062
1LB 0I 2RG 0.125
1LB 0I 3RG 0.375
1LB 1I 2RG 0.375
1LB 1I 3RG 0.062
Non-tank rotations %: 0
Rotating on tank?: Yes
Rotations: 1LB 0I 3RG (0.375), 1LB 1I 2RG (0.375)
Removing spreadsheet duplicates..
Duplicate removal complete!
-------
459 Hotspotato  Whitemane
Lifebloom uptime %: 89.69%
1LB 3I 0RG 0.211
1LB 4I 0RG 0.211
1LB 1I 3RG 0.053
1LB 3I 1RG 0.105
1LB 0I 3RG 0.053
1LB 2I 2RG 0.105
0LB 1I 3RG 0.053
1LB 2I 1RG 0.158
1LB 1I 1RG 0.053
Non-tank rotations %: 0.05
Rotating on tank?: Yes
Rotations: 1LB 3I 0RG (0

Removing spreadsheet duplicates..
Duplicate removal complete!
-------
484 桶面  哈霍兰
Lifebloom uptime %: 0
0LB 5I 0RG 0.062
0LB 1I 4RG 0.438
0LB 2I 3RG 0.188
0LB 0I 5RG 0.312
Non-tank rotations %: 1.0
Rotating on tank?: No
Rotations: 0LB 1I 4RG (0.438), 0LB 0I 5RG (0.312)
Removing spreadsheet duplicates..
Duplicate removal complete!
-------
485 Facîes Firemaw (EU)
Lifebloom uptime %: 90.12%
1LB 2I 0RG 0.056
1LB 3I 0RG 0.111
1LB 1I 2RG 0.056
1LB 0I 4RG 0.111
1LB 1I 1RG 0.056
1LB 2I 2RG 0.111
1LB 3I 1RG 0.111
0LB 5I 0RG 0.111
0LB 1I 4RG 0.056
2LB 0I 3RG 0.056
1LB 0I 2RG 0.056
1LB 0I 3RG 0.056
0LB 3I 0RG 0.056
Non-tank rotations %: 0.22
Rotating on tank?: Yes
Rotations: 1LB 3I 0RG (0.111), 1LB 0I 4RG (0.111)
Removing spreadsheet duplicates..
Duplicate removal complete!
-------
486 我是大坏蛋  狮心
Lifebloom uptime %: 42.69%
1LB 0I 3RG 0.071
1LB 0I 4RG 0.071
0LB 1I 4RG 0.214
0LB 0I 5RG 0.429
1LB 1I 3RG 0.071
0LB 2I 3RG 0.143
Non-tank rotations %: 0.79
Rotating on tank?: No
Rotations: 0LB 0I 5RG (0.4